In [15]:
import sympy
from sympy import symbols, factorial, Function, summation, binomial, product, Sum

n, i, j, m = symbols('n, i, j, m', integer=True, real=True)

t = symbols('t')
T = symbols('T')
beta = sympy.symbols('beta')
beta_dot = 1/T
P = Function('P')

def bernstein(i, n, t):
    return binomial(n, i)*(1 - t)**(n-i)*t**i

def bezier(i, n, t):
    return Sum(bernstein(i, n, t)*P(i), (i, 0, n))

def bezier_dot(i, n, t):
    return Sum(bernstein(i, n-1, t)*(P(i+1) - P(i)), (i, 0, n-1))

In [16]:
Sum(i, (i, 0, 3)).doit()

6

In [17]:
bezier(i, n, t)

Sum(t**i*(1 - t)**(-i + n)*P(i)*binomial(n, i), (i, 0, n))

In [18]:
bezier_dot(i, n, t)

Sum(t**i*(1 - t)**(-i + n - 1)*(-P(i) + P(i + 1))*binomial(n - 1, i), (i, 0, n - 1))

In [19]:
f_P = sympy.Function('P')
n = 8

D = [[ f_P(i) for i in range(n) ]]

for j in range(1, n-1):
    print(j)
    Dj = [ D[j-1][i+1] - D[j-1][i] for i in range(n-j-1) ]
    D.append(Dj)
    print(Dj)
D


1
[-P(0) + P(1), -P(1) + P(2), -P(2) + P(3), -P(3) + P(4), -P(4) + P(5), -P(5) + P(6)]
2
[P(0) - 2*P(1) + P(2), P(1) - 2*P(2) + P(3), P(2) - 2*P(3) + P(4), P(3) - 2*P(4) + P(5), P(4) - 2*P(5) + P(6)]
3
[-P(0) + 3*P(1) - 3*P(2) + P(3), -P(1) + 3*P(2) - 3*P(3) + P(4), -P(2) + 3*P(3) - 3*P(4) + P(5), -P(3) + 3*P(4) - 3*P(5) + P(6)]
4
[P(0) - 4*P(1) + 6*P(2) - 4*P(3) + P(4), P(1) - 4*P(2) + 6*P(3) - 4*P(4) + P(5), P(2) - 4*P(3) + 6*P(4) - 4*P(5) + P(6)]
5
[-P(0) + 5*P(1) - 10*P(2) + 10*P(3) - 5*P(4) + P(5), -P(1) + 5*P(2) - 10*P(3) + 10*P(4) - 5*P(5) + P(6)]
6
[P(0) - 6*P(1) + 15*P(2) - 20*P(3) + 15*P(4) - 6*P(5) + P(6)]


[[P(0), P(1), P(2), P(3), P(4), P(5), P(6), P(7)],
 [-P(0) + P(1),
  -P(1) + P(2),
  -P(2) + P(3),
  -P(3) + P(4),
  -P(4) + P(5),
  -P(5) + P(6)],
 [P(0) - 2*P(1) + P(2),
  P(1) - 2*P(2) + P(3),
  P(2) - 2*P(3) + P(4),
  P(3) - 2*P(4) + P(5),
  P(4) - 2*P(5) + P(6)],
 [-P(0) + 3*P(1) - 3*P(2) + P(3),
  -P(1) + 3*P(2) - 3*P(3) + P(4),
  -P(2) + 3*P(3) - 3*P(4) + P(5),
  -P(3) + 3*P(4) - 3*P(5) + P(6)],
 [P(0) - 4*P(1) + 6*P(2) - 4*P(3) + P(4),
  P(1) - 4*P(2) + 6*P(3) - 4*P(4) + P(5),
  P(2) - 4*P(3) + 6*P(4) - 4*P(5) + P(6)],
 [-P(0) + 5*P(1) - 10*P(2) + 10*P(3) - 5*P(4) + P(5),
  -P(1) + 5*P(2) - 10*P(3) + 10*P(4) - 5*P(5) + P(6)],
 [P(0) - 6*P(1) + 15*P(2) - 20*P(3) + 15*P(4) - 6*P(5) + P(6)]]

In [20]:
n, m, j = sympy.symbols('n, m, j', integer=True)
sympy.Product((n - m), (m, 0, j-1))

Product(-m + n, (m, 0, j - 1))

In [21]:
C_j = sympy.Product((n - m), (m, 0, j-1))*\
    sympy.Sum((-1)**(i + j)*f_P(i)/(sympy.factorial(i)*sympy.factorial(j - i)), (i, 0, j))
C_j

Product(-m + n, (m, 0, j - 1))*Sum((-1)**(i + j)*P(i)/(factorial(i)*factorial(-i + j)), (i, 0, j))

In [37]:
C_j = C_j.doit()
C_j

(-1)**j*RisingFactorial(-n, j)*Sum((-1)**(i + j)*P(i)/(factorial(i)*factorial(-i + j)), (i, 0, j))

In [40]:
from sympy.abc import x, y, z
from sympy.utilities.codegen import codegen

[(c_name, c_code), (h_name, c_header)] = codegen(("C_j", sympy.product(i, 0, i)), "C89", "test", header=False, empty=False)

ValueError: Invalid limits given: (0, i)

In [39]:
print(c_code)

#include "test.h"
#include <math.h>
double C_j(int j, int n) {
   double C_j_result;
   C_j_result = pow(-1, j)*RisingFactorial(-n, j)*Sum(pow(-1, i + j)*P(i)/(factorial(i)*factorial(-i + j)), (i, 0, j));
   return C_j_result;
}



In [23]:
[ C_j.subs(j, ji).subs(n, 7).doit() for ji in range(7)]

[P(0),
 -7*P(0) + 7*P(1),
 21*P(0) - 42*P(1) + 21*P(2),
 -35*P(0) + 105*P(1) - 105*P(2) + 35*P(3),
 35*P(0) - 140*P(1) + 210*P(2) - 140*P(3) + 35*P(4),
 -21*P(0) + 105*P(1) - 210*P(2) + 210*P(3) - 105*P(4) + 21*P(5),
 7*P(0) - 42*P(1) + 105*P(2) - 140*P(3) + 105*P(4) - 42*P(5) + 7*P(6)]

In [217]:
sympy.Product((n - m), (m, 0, j-1)).doit()

(-1)**j*RisingFactorial(-n, j)